In [27]:
from fastcore.all import *
import wandb
from wandb.keras import WandbCallback

In [28]:
kitti_path = Path("../dataset_samples/sample_dataset/")
kitti_path.exists()

True

In [29]:
import os.path

import argparse
import keras
from data_loader import DataLoader
from utils.callbacks import TensorBoard
from utils.util import *
from utils.args_loader import load_model_config

In [30]:
import tensorflow as tf

In [31]:
tf.config.run_functions_eagerly(True)

In [32]:
arg = SimpleNamespace(model="squeezesegv2",
                         config="squeezesegv2",
                         data_path=kitti_path,
                         train_dir="../output",
                         epochs=10)
                         

In [33]:
config, model = load_model_config(arg.model, arg.config)

In [34]:
train_dl = DataLoader("train", arg.data_path, config).write_tfrecord_dataset().read_tfrecord_dataset()
val_dl = DataLoader("val", arg.data_path, config).write_tfrecord_dataset().read_tfrecord_dataset()

TFRecord exists at ../dataset_samples/sample_dataset/train.tfrecord. Skipping TFRecord writing.
TFRecord exists at ../dataset_samples/sample_dataset/val.tfrecord. Skipping TFRecord writing.


In [35]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=config.LEARNING_RATE,
    decay_steps=config.LR_DECAY_STEPS,
    decay_rate=config.LR_DECAY_FACTOR,
    staircase=True)

## Let's log some data

In [36]:
import tensorflow as tf
import numpy
import keras


class LogSamplesCallback(WandbCallback):
    "A simple Keras callback to log model predictions"
    
    def __init__(self, dataset, **kwargs):
        super().__init__(**kwargs)
        self.dataset = dataset
        self.num_images = 1
        
    def on_epoch_end(self, epoch, logs=None):
        super().on_epoch_end(epoch, logs=logs)
        
        # get first batch of dataset
        class_color_map = self.model.CLS_COLOR_MAP
        (lidar_input, lidar_mask), label, weight = self.dataset.take(1).get_single_element()

        probabilities, predictions = self.model([lidar_input, lidar_mask])

        label = label[:self.num_images, :, :]
        weight = weight[:self.num_images, :, :].numpy()
        predictions = predictions[:self.num_images, :, :].numpy()

        # label and prediction visualizations
        label_image = class_color_map[label.numpy().reshape(-1)].reshape([self.num_images, label.shape[1], label.shape[2], 3])
        pred_image = class_color_map[predictions.reshape(-1)].reshape([self.num_images, label.shape[1], label.shape[2], 3])
        weight_image = weight.reshape([self.num_images, weight.shape[1], weight.shape[2], 1])
        depth_image = lidar_input.numpy()[:self.num_images, :, :, [4]]
        intensity = lidar_input.numpy()[:self.num_images, :, :, [3]]
        points = lidar_input.numpy()[:self.num_images, :, :, :3]
        points_rgb = np.concatenate([points, (255*label_image).astype(int)], axis=-1).reshape(-1, 6)
        
        def _resize(img, factor=3):
            return tf.image.resize(img, [img.shape[1]*factor, img.shape[2]*factor]).numpy()
        intensity_image, depth_image, weight_image,label_image, pred_image = map(_resize, [intensity, depth_image, weight_image,label_image, pred_image])
        
        # log a bunch of images
        wandb.log({'Images/Label Image': wandb.Image(label_image)}, step=epoch)
        wandb.log({'Images/Depth Image': wandb.Image(depth_image)}, step=epoch)
        wandb.log({'Images/Intensity Image': wandb.Image(intensity_image)}, step=epoch)
        wandb.log({'Images/Prediction Image':wandb.Image(pred_image)}, step=epoch)
        wandb.log({"Images/3D": wandb.Object3D({"type": "lidar/beta", "points":points_rgb})})


In [76]:
val_dl

<PrefetchDataset element_spec=((TensorSpec(shape=(1, 32, 240, 6), dtype=tf.float32, name=None), TensorSpec(shape=(1, 32, 240), dtype=tf.bool, name=None)), TensorSpec(shape=(1, 32, 240), dtype=tf.int32, name=None), TensorSpec(shape=(1, 32, 240), dtype=tf.float32, name=None))>

In [75]:
next(iter(val_dl))

((<tf.Tensor: shape=(1, 32, 240, 6), dtype=float32, numpy=
  array([[[[ 0.        ,  0.        ,  0.        ,  0.        ,
             0.        ,  0.        ],
           [ 0.        ,  0.        ,  0.        ,  0.        ,
             0.        ,  0.        ],
           [ 0.        ,  0.        ,  0.        ,  0.        ,
             0.        ,  0.        ],
           ...,
           [ 0.01911327, -1.406345  ,  3.5605857 , -0.21036972,
             0.09308348,  1.        ],
           [-0.0066227 , -1.379111  ,  3.4559512 , -0.01201999,
             0.06615528,  1.        ],
           [ 0.        ,  0.        ,  0.        ,  0.        ,
             0.        ,  0.        ]],
  
          [[ 0.35335293,  1.9470439 ,  3.452807  ,  0.0673199 ,
             0.45894414,  1.        ],
           [ 0.26908162,  1.7816892 ,  3.1993546 ,  0.30533957,
             0.3648252 ,  1.        ],
           [ 0.2209083 ,  1.6803578 ,  3.0525827 ,  0.18632974,
             0.31032145,  1.     

In [71]:
val_dl.take(1).get_single_element()

((<tf.Tensor: shape=(1, 32, 240, 6), dtype=float32, numpy=
  array([[[[ 1.009395  ,  3.0777023 ,  7.937118  ,  0.741709  ,
             1.2126087 ,  1.        ],
           [ 1.0246376 ,  3.073608  ,  7.9894347 ,  0.7813789 ,
             1.2263048 ,  1.        ],
           [ 1.0340153 ,  3.059612  ,  8.017606  ,  0.7813789 ,
             1.2335445 ,  1.        ],
           ...,
           [-0.20240848, -1.0604539 ,  2.6259193 , -0.32937956,
            -0.14943846,  1.        ],
           [-0.22895499, -1.0291561 ,  2.5167575 , -0.4483894 ,
            -0.17752884,  1.        ],
           [ 0.        ,  0.        ,  0.        ,  0.        ,
             0.        ,  0.        ]],
  
          [[ 0.7972451 ,  2.7089434 ,  4.7768946 ,  0.5036893 ,
             0.9434718 ,  1.        ],
           [ 0.7750833 ,  2.6445308 ,  4.70438   ,  0.42434943,
             0.9165325 ,  1.        ],
           [ 0.        ,  0.        ,  0.        ,  0.        ,
             0.        ,  0.     

In [37]:
(lidar_input, lidar_mask), label, weight = val_dl.take(1).get_single_element()

In [38]:
class_color_map = model.CLS_COLOR_MAP

In [39]:
num_images=1

In [40]:
label_image = class_color_map[label.numpy().reshape(-1)].reshape([num_images, label.shape[1], label.shape[2], 3])

In [41]:
points = lidar_input[..., :3].numpy()

In [42]:
points.shape

(1, 32, 240, 3)

In [43]:
label_image.shape

(1, 32, 240, 3)

In [44]:
a = (label_image*255)

In [45]:
a.astype(int)

array([[[[  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         ...,
         [107, 142,  35],
         [107, 142,  35],
         [  0,   0,   0]],

        [[107, 142,  35],
         [107, 142,  35],
         [107, 142,  35],
         ...,
         [107, 142,  35],
         [107, 142,  35],
         [  0,   0,   0]],

        [[107, 142,  35],
         [107, 142,  35],
         [107, 142,  35],
         ...,
         [107, 142,  35],
         [  0,   0,   0],
         [  0,   0,   0]],

        ...,

        [[128,  64, 128],
         [128,  64, 128],
         [128,  64, 128],
         ...,
         [128,  64, 128],
         [128,  64, 128],
         [128,  64, 128]],

        [[128,  64, 128],
         [128,  64, 128],
         [128,  64, 128],
         ...,
         [128,  64, 128],
         [128,  64, 128],
         [128,  64, 128]],

        [[  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         ...,
         [  0,   0,   0],
        

In [46]:
points_rgb = np.concatenate([points, label_image], axis=-1).reshape(-1, 6)

In [47]:
points_rgb.shape

(7680, 6)

In [48]:
wandb.Object3D({"type": "lidar/beta", "points":points_rgb})
    

In [49]:
wandb.init(project="lidar_tf", entity="av-team", sync_tensorboard=True)

wandb_callback = LogSamplesCallback(dataset=val_dl, save_model=False)
# tensorboard_callback = TensorBoard(arg.train_dir, val_dl, profile_batch=(200, 202))
# checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(os.path.join(arg.train_dir, "checkpoint"))

# cbs = [wandb_callback, tensorboard_callback, checkpoint_callback]
cbs = [wandb_callback]

In [50]:
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule, clipnorm=config.MAX_GRAD_NORM)

model.compile(optimizer=optimizer, weighted_metrics=[])

In [51]:
model.fit(train_dl,
        validation_data=val_dl,
        epochs=arg.epochs,
        callbacks = cbs,
        )

wandb: WARNING Unable to compute FLOPs for this model.


Epoch 1/10
1/1 [==============================] - 4s 4s/step - loss: 2.4020 - miou: 0.0865 - val_loss: 2.2477 - val_miou: 0.0455
Epoch 2/10
1/1 [==============================] - 4s 4s/step - loss: 2.1089 - miou: 0.1170 - val_loss: 2.1452 - val_miou: 0.0446
Epoch 3/10
1/1 [==============================] - 4s 4s/step - loss: 1.8016 - miou: 0.1253 - val_loss: 2.2887 - val_miou: 0.0403
Epoch 4/10
1/1 [==============================] - 4s 4s/step - loss: 1.3857 - miou: 0.1061 - val_loss: 8.7842 - val_miou: 0.0207
Epoch 5/10
1/1 [==============================] - 4s 4s/step - loss: 3.3396 - miou: 0.0883 - val_loss: 2.7583 - val_miou: 0.0682
Epoch 6/10
1/1 [==============================] - 4s 4s/step - loss: 1.0957 - miou: 0.1699 - val_loss: 2.4551 - val_miou: 0.0878
Epoch 7/10
1/1 [==============================] - 4s 4s/step - loss: 1.1601 - miou: 0.2111 - val_loss: 2.4437 - val_miou: 0.0914
Epoch 8/10
1/1 [==============================] - 4s 4s/step - loss: 1.1898 - miou: 0.2323 - val_

In [52]:
wandb.finish()

epoch,▁▂▃▃▄▅▆▆▇█
loss,▅▄▃▂█▁▁▁▁▁
miou,▁▂▂▂▁▄▆▆██
val_loss,▁▁▁█▂▁▁▁▁▁
val_miou,▃▃▃▁▅▇▇███
best_epoch,1
best_val_loss,2.14516
epoch,9
loss,1.16002
miou,0.27662
val_loss,2.5905
